In [ ]:
!pip install torch

     |███████████████████████████████ | 78.6MB 80kB/s eta 0:00:312

In [ ]:
import torch
import torch.nn as nn
from tensorflow import keras
import torch.nn.functional as F
from torch import optim

In [ ]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
batch_size = 64
trn_loader = torch.utils.data.DataLoader(train_images,
                                         batch_size = batch_size,
                                         shuffle = True)

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(28*28, 300)
        self.fc2 = nn.Linear(300, 128)
    
        self.mean = nn.Linear(128, 10)
        self.std = nn.Linear(128,10)
        

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)

        mean = self.mean(x)
        std = torch.exp(self.std(x))
        
        return mean, std

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.Dense1 = nn.Linear(10, 40)
        self.Dense2 = nn.Linear(40, 128)
        self.Dense3 = nn.Linear(128, 784)
        
    def forward(self, x):
        x = self.Dense1(x)
        x = F.relu(x)
        x = self.Dense2(x)
        x = F.relu(x)
        x = self.Dense3(x)
        x = F.relu(x)
        
        return x.reshape([28, 28])

In [ ]:
learning_rate = 0.01
num_epochs = 2
num_batches = len(trn_loader)
normal_sampler = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0]))


def train(encoder, decoder):
    encoder_optimizer = optim.Adam(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr = learning_rate)
    
    for it in range(num_epochs):
        loss = 0

        for i, data in enumerate(trn_loader):
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            
            mean, std = encoder(data.view(-1,1,28*28).double())
            norm = normal_sampler.sample(sample_shape=(std.size(0),1)).view(1,-1)
            
            decoder_input = torch.add(mean, torch.mul(std, norm))
            output = decoder(output)
            
            loss += torch.dist(data, output)
            
        loss.backward(retain_graph=True)

        encoder_optimizer.step()
        decoder_optimizer.step()
    
    return mean, std

In [ ]:
encoder = Encoder()
decoder = Decoder()
mean, std = train(encoder, decoder)